In [11]:
# Settings
import os
import json
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # 空间三维画图

# Setting for phoenix
# w = 210
# h = 260
# Setting for CSL
w = 720
h = 800
num_joints = 137
view_joints = [0,1,2,3,4,5,6,7,8,9,12,15,16,17,18]
face_view_joints = numpy.arange(25,75)
lefhand_view_joints = numpy.arange(75,96)
righthand_view_joints = numpy.arange(96,137)
view_joints = numpy.concatenate([view_joints,face_view_joints,lefhand_view_joints,righthand_view_joints],0)
# [
#     [4,3],
#     [6,7]
# ]
line_pairs = numpy.array(\
[
    [17,15],
    [15,0],
    [0,16],
    [16,18],
    [0,1],
    [2,1],
    [1,5],
    [1,8],
    [4,3],
    [3,2],
    [5,6],
    [6,7],
    [9,8],
    [8,12]
])
hand_pairs = numpy.array(\
[
    [0,1],
    [1,2],
    [2,3],
    [3,4],
    [0,5],
    [5,6],
    [6,7],
    [7,8],
    [0,9],
    [9,10],
    [10,11],
    [11,12],
    [0,13],
    [13,14],
    [14,15],
    [15,16],
    [0,17],
    [17,18],
    [18,19],
    [19,20]
])
lefthand_pairs = hand_pairs + 95
righthand_pairs = hand_pairs + 116
line_pairs = numpy.concatenate([line_pairs,lefthand_pairs,righthand_pairs],0)
# Colors
lightred = [0.93,0.60,0.60]
red = [0.83,0.18,0.18]
orange = [1.0,0.5,0.2]
blue = [0.0,0.66,0.88]
pink = [0.92,0.38,0.57]
purple = [0.70,0.53,1.0]
yellow = [1.0,0.93,0.35]

def read_json(file):
    skeletonDict = json.load(open(file,'r'))
    bodySkeleton = numpy.array(skeletonDict['Body']).squeeze()
    faceSkeleton = numpy.array(skeletonDict['Face']).squeeze()
    leftHandSkeleton = numpy.array(skeletonDict['Left hand']).squeeze()
    rightHandSkeleton = numpy.array(skeletonDict['Right hand']).squeeze()
    skeleton = numpy.concatenate([bodySkeleton, faceSkeleton, leftHandSkeleton, rightHandSkeleton],-2)
    if len(skeleton.shape)>2:
        skeleton = skeleton[0]
    return skeleton[:,:2]

def get_pairs(skeleton, line_pairs):
    pairs = []
    for pair in line_pairs:
        i = pair[0]
        j = pair[1]
        if (skeleton[i]==0).all() or (skeleton[j]==0).all():
            print(pair)
        else:
            pairs.append(pair)
    return numpy.array(pairs)

def get_colors():
    # Paint diffenrent color
    colors = []
    for i in range(num_joints):
        if i in [0,15,16,17,18]:
            color = red
        elif i in [1,8,9,12]:
            color = orange
        elif i in [2,3,4]:
            color = blue
        elif i in [5,6,7]:
            color = pink
        elif 25<=i and i<=94:
            color = lightred
        elif 95<=i and i<=115:
            color = purple
        elif 116<=i and i<=136:
            color = yellow
        colors.append(color)
    return numpy.array(colors)


In [ ]:
# generate from json file
skeleton_folder = '/Users/liweijie/SLR/scripts/01April_2010_Thursday_heute_default-0'
file_list = os.listdir(skeleton_folder)
file_list.sort()
mat = []
point_indices = []
line_indices = []
for i,file in enumerate(file_list[::20]):
    file_path = os.path.join(skeleton_folder,file)
    skeleton = read_json(file_path)
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    t = 20*10*(i/len(file_list))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)
    line_indices_t = i*num_joints+get_pairs(skeleton,line_pairs)
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)
mat = numpy.stack(mat,0)
point_indices = numpy.stack(point_indices,0)
line_indices = numpy.stack(line_indices,0)

# Save as byte data
mat.astype(numpy.float32).tofile('/Users/liweijie/projects/visualize-skeleton/skeleton.data')
point_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/point_indices.data')
line_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/line_indices.data')

In [12]:
import numpy
# generate from npy
npy_file = 'vae_generate/input_000000.npy'
# Shape of data is N x L(32) x J x D
data = numpy.load(npy_file)
data = data[0]
mat = []
point_indices = []
line_indices = []
for i,skeleton in enumerate(data):
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    t = 2*10*(i/len(data))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)
    line_indices_t = i*num_joints+line_pairs
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)
mat = numpy.stack(mat,0)
point_indices = numpy.stack(point_indices,0)
line_indices = numpy.stack(line_indices,0)

# Save as byte data
mat.astype(numpy.float32).tofile('/Users/liweijie/projects/visualize-skeleton/skeleton.data')
point_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/point_indices.data')
line_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/line_indices.data')

In [7]:
data1 = numpy.load('vae_generate/input_000000.npy')
data2 = numpy.load('vae_generate/recons_000000.npy')
data1,data2

(array([[[[-0.1375846 , -0.87978685],
          [-0.04289101, -0.48167276],
          [-0.6690446 , -0.48186815],
          ...,
          [-0.9376585 ,  0.85922056],
          [-0.8753171 ,  0.8927394 ],
          [-0.83695334,  0.9284929 ]],
 
         [[-0.12437606, -0.8700666 ],
          [-0.03033898, -0.4651626 ],
          [-0.6516105 , -0.4735656 ],
          ...,
          [-0.93342847,  0.8686937 ],
          [-0.87636733,  0.8954    ],
          [-0.8288164 ,  0.93323386]],
 
         [[-0.10049409, -0.8849742 ],
          [-0.00162787, -0.45705038],
          [-0.5469804 , -0.45728612],
          ...,
          [-0.95375884,  0.84396076],
          [-0.9383447 ,  0.89190364],
          [-0.8921035 ,  0.9169172 ]],
 
         ...,
 
         [[-0.14238986, -0.8692104 ],
          [-0.06559936, -0.46556264],
          [-0.69719297, -0.46544826],
          ...,
          [-0.9322247 ,  0.8662981 ],
          [-0.8734865 ,  0.89600956],
          [-0.8554127 ,  0.9172321 ]],
 


In [10]:
mat.shape,point_indices.shape,line_indices.shape

((9, 137, 6), (9, 127), (9, 54, 2))

In [7]:
point_indices[0],line_indices[0]

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  12,  15,  16,
         17,  18,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
         36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
         49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
         62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
         75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
         88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
        101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
        114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
        127, 128, 129, 130, 131, 132, 133, 134, 135, 136]),
 array([[ 17,  15],
        [ 15,   0],
        [  0,  16],
        [ 16,  18],
        [  0,   1],
        [  2,   1],
        [  1,   5],
        [  1,   8],
        [  4,   3],
        [  3,   2],
        [  5,   6],
        [  6,   7],
        [  9,   8],
        [  8,  12],
   

In [3]:
import matplotlib.pyplot as plt
def plot_skeleton(skeleton):
    mat = skeleton
    bodySkeleton = skeleton[0:25]
    faceSkeleton = skeleton[25:95]
    leftHandSkeleton = skeleton[95:116]
    rightHandSkeleton = skeleton[116:137]
    plt.scatter(bodySkeleton[:,0],-bodySkeleton[:,1])
    plt.scatter(mat[[3,4,6,7],0],-mat[[3,4,6,7],1])
    plt.scatter(faceSkeleton[:,0],-faceSkeleton[:,1])
    plt.scatter(leftHandSkeleton[:,0],-leftHandSkeleton[:,1])
    plt.scatter(rightHandSkeleton[:,0],-rightHandSkeleton[:,1])

    plt.show()